Para poner en práctica nuestros amplios conocimientos recién adquiridos de Spark, realizaremos un caso práctico en el mundo financiero.  
  
En este caso, nos encontramos con dos tablas. Una contiene nuestros clientes. La otra sus datos de tarjeta.  
  
A lo largo del cuaderno se te irá guiando con las acciones que deberás realizar en cada celda.

#### Inicia el contexto de Spark
Para esta tarea, se pide que la aplicación de Spark se llame 'JugandoASerBanquero' y la aplicación se ejecute en tantos worker nodes como núcleos disponibles hay en nuestra máquina.

In [1]:
import pyspark

conf = pyspark.SparkConf().setAppName('JugandoASerBanquero').setMaster('local[*]') #Creamos la configuración
sc = pyspark.SparkContext(conf = conf) #Abrimos el contexto de Spark

In [2]:
import pyspark.sql

sparkSession = pyspark.sql.SparkSession(sc, jsparkSession=None)

spark = sparkSession\
.builder\
.master("local")\
.appName("holi")\
.config("spark.some.config.option", "some-value")\
.getOrCreate()

#### Lee los datos de clientes y tarjetas
Estos datos están almacenados en dos archivos .txt, uno para **clientes** y otro para **tarjetas** dentro del directorio */data*.  
Carga ambos datasets como RDDs en el contexto de Spark y muestra todos los clientes cargados. En cuanto a las tarjetas, muestra cúantas tarjetas hay, y aleatoriamente muestra por pantalla un número menor a las tarjetas existentes, sin reemplazamiento.

In [3]:
clientesRDD = sc.textFile("/home/afernandez/jupyter-spark/data/clientes.txt")
tarjetasRDD = sc.textFile("/home/afernandez/jupyter-spark/data/tarjetas.txt")

In [4]:
clientesRDD.collect()

['80000000,Antonio Lopez Ramirez,Calle Cantalapiedra 8',
 '70000000,Francisco Arias Sanchez,Avenida de America 12',
 '50000000,Norberto Marias Quintana,Calle Uganda 88',
 '10000000,Julio Cortazar Carter,Calle Bruselas 14',
 '20000000,Arturo Belano Yañez,Travesia de Calvo Sotelo 13']

In [5]:
tarjetasRDD.count()

5

In [6]:
tarjetasRDD.takeSample(False, 3)

['20000000,Arturo Belano Yañez,American Express,1111222233334444',
 '50000000,Norberto Marias Quintana,Visa,1001100110011001',
 '10000000,Julio Cortazar Carter,Master Card,2003200320032003']

#### Crear un archivo Clases.py
En este archivo crea dos clases,
1. ***Cliente***, la cual tiene estos atributos:
    * DNI
    * Nombre
    * Direccion
2. ***Tarjeta***, con los siguientes atributos:
    * DNI
    * Nombre
    * Tipo de tarjeta
    * Número de tarjeta  

Como puedes observar en la carga de datos, el formato de los datos es un string por cliente o tarjeta, en el cual cada atributo está separado por una coma ",".  
Con tal de cargar los datos en los atributos de cada clase, define convenientemente la asignación de los elementos en el método \__init__ de cada clase.

In [7]:
class Cliente:
    def __init__(self, linea):
        self.dni, self.nombre, self.direccion = linea.split(',')
class Tarjeta(object):
    def __init__(self, linea):
        self.dni, self.nombre, self.tipo_tarjeta, self.num_tarjeta = linea.split(',')

#### Transforma los datos de clientes y tarjetas obtenidos
A continuación, crea los distintos objetos de clientes y tarjetas importados anteriormente. *Utiliza en al menos uno de ellos una **función lambda**.*

In [9]:
from clases import Cliente, Tarjeta

objClientesRDD = clientesRDD.map(Cliente)
objTarjetasRDD = tarjetasRDD.map(lambda x: Tarjeta(x))

Devuelve el primer cliente almacenado

In [10]:
objClientesRDD.first()

#### Agrupa los clientes y tarjetas en un mismo RDD
Ten en cuenta que se necesita un pairRDD para cruzar datasets.

In [13]:
# Tabla = objClientesRDD.join(objTarjetasRDD) --> Necesitamos clave-valor
joinRDD = objClientesRDD.map(lambda x: (x.dni, x)).join(objTarjetasRDD.map(lambda x: (x.dni, x)))

La estructura devuelta tiene la forma de un pair RDD,
* **key**: DNI
* **value**: Información completa del cliente y tarjeta en forma de tuple

In [14]:
joinRDD.values().collect()

[(<clases.Cliente at 0x7ff019a99668>, <clases.Tarjeta at 0x7ff019a997f0>),
 (<clases.Cliente at 0x7ff019a99780>, <clases.Tarjeta at 0x7ff019a99898>),
 (<clases.Cliente at 0x7ff019a99748>, <clases.Tarjeta at 0x7ff019a99908>),
 (<clases.Cliente at 0x7ff019a997b8>, <clases.Tarjeta at 0x7ff019a999b0>),
 (<clases.Cliente at 0x7ff019a99860>, <clases.Tarjeta at 0x7ff019a99a58>)]

In [15]:
ClientesTarjetas = joinRDD.map(lambda x: x[1][0].nombre + ' - ' + x[1][1].tipo_tarjeta + ' - ' + x[1][1].num_tarjeta)
ClientesTarjetas.collect()

['Arturo Belano Yañez - American Express - 9008900890089008',
 'Arturo Belano Yañez - American Express - 1111222233334444',
 'Francisco Arias Sanchez - Visa - 1200120012001200',
 'Norberto Marias Quintana - Visa - 1001100110011001',
 'Julio Cortazar Carter - Master Card - 2003200320032003']

In [16]:
print(joinRDD.collect()[2][0]) # Clave del tercer elemento
print(joinRDD.collect()[1][1]) # Valor del segundo elemento

70000000
(<clases.Cliente object at 0x7ff019a88f60>, <clases.Tarjeta object at 0x7ff019cc4710>)


#### Devuelve los datos cruzados
Se pide que se devuelvan los datos con este formato.  

*Nombre - Tipo de tarjeta - Número de tarjeta*

Como vemos, al cruzar ambas tablas hay clientes que tienen más de una tarjeta, y por ello aparecen duplicados en la tabla de ClientesTarjetas. Por ello, a continuación agrupa las tarjetas y clientes únicos, de manera que obtengas un pairRDD con la siguiente estructura:
* **key**: Cliente
* **value**: Tarjetas, en forma de lista

Debe quedar un RDD de estructura similar a:  
***\[(Cliente1, [Tarjeta1, Tarjeta2]),  
(Cliente2, [Tarjeta1, Tarjeta2, Tarjeta3]),  
(Cliente3, [Tarjeta1])\]***

In [23]:
GroupedByDNI = joinRDD.groupByKey()
GroupedByDNI.collect()

[('20000000', <pyspark.resultiterable.ResultIterable at 0x7f8c40acef98>),
 ('70000000', <pyspark.resultiterable.ResultIterable at 0x7f8c40ace390>),
 ('50000000', <pyspark.resultiterable.ResultIterable at 0x7f8c40ace160>),
 ('10000000', <pyspark.resultiterable.ResultIterable at 0x7f8c40ace0f0>)]

Como se observa en la anterior ejecución, los valores del RDD GroupedByDNI son objetos iterables (que no podemos recorrer mediante un for), en cambio podemos mapear los valores mediante la función lista.

In [232]:
GroupedByDNI = joinRDD.groupByKey().mapValues(list)
# GroupedByDNI = Finaljoin.map(lambda x: (x[0],list(x[1]))) # Es equivalente a la anterior linea
GroupedByDNI.collect()

[('20000000',
  [(<clases.Cliente at 0x7f5bb02aada0>, <clases.Tarjeta at 0x7f5bb0266b38>),
   (<clases.Cliente at 0x7f5bb02aada0>, <clases.Tarjeta at 0x7f5bb0266278>)]),
 ('70000000',
  [(<clases.Cliente at 0x7f5bb0266438>, <clases.Tarjeta at 0x7f5bb0266b00>)]),
 ('50000000',
  [(<clases.Cliente at 0x7f5bb0266c88>, <clases.Tarjeta at 0x7f5bb0266630>)]),
 ('10000000',
  [(<clases.Cliente at 0x7f5bb0266400>, <clases.Tarjeta at 0x7f5bb02663c8>)])]

In [71]:
def ExtractCards(x):
    value = []
    for elemento in x[1]:
        key = elemento[0]
        key = key.nombre
        valor = elemento[1]
        value.append(valor)
    return (key,value)

In [72]:
ClientesTarjetas = GroupedByDNI.map(ExtractCards)
ClientesTarjetas.collect()

[('Arturo Belano Yañez',
  [<clases.Tarjeta at 0x7f8c40ad5470>, <clases.Tarjeta at 0x7f8c40ae2240>]),
 ('Francisco Arias Sanchez', [<clases.Tarjeta at 0x7f8c40ae2be0>]),
 ('Norberto Marias Quintana', [<clases.Tarjeta at 0x7f8c40ae2e10>]),
 ('Julio Cortazar Carter', [<clases.Tarjeta at 0x7f8c40ae2320>])]

In [20]:
ClientesTarjetas = joinRDD.map(lambda x: x[1][0].dni + ',' + x[1][0].nombre + ',' + x[1][0].direccion + ',' + x[1][1].tipo_tarjeta + ',' + x[1][1].num_tarjeta)
ClientesTarjetas.saveAsTextFile("/home/afernandez/jupyter-spark/data/bankcards")

PART 2

In [39]:
from pyspark.sql.types import *
schemaCards = (StructType([
    StructField("DNI", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Address", StringType(), True),
    StructField("CardType", StringType(), True),
    StructField("CardNumber", LongType(), True)]))

clientscards = spark.read.load("/home/afernandez/jupyter-spark/data/bankcard.csv",
                     format="csv", sep=",", header="true", schema = schemaCards)

clientscards.show()

+--------+--------------------+--------------------+----------------+----------------+
|     DNI|                Name|             Address|        CardType|      CardNumber|
+--------+--------------------+--------------------+----------------+----------------+
|20000000| Arturo Belano Yañez|Travesia de Calvo...|American Express|9008900890089010|
|20000000| Arturo Belano Yañez|Travesia de Calvo...|American Express|1111222233334444|
|70000000|Francisco Arias S...|Avenida de Americ...|            Visa|1200120012001200|
|50000000|Norberto Marias Q...|     Calle Uganda 88|            Visa|1001100110011001|
|10000000|Julio Cortazar Ca...|   Calle Bruselas 14|     Master Card|2003200320032003|
+--------+--------------------+--------------------+----------------+----------------+



In [55]:
clientscards.printSchema()

root
 |-- DNI: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- CardType: string (nullable = true)
 |-- CardNumber: long (nullable = true)



In [49]:
VISA = clientscards.filter(clientscards['CardType'] == 'Visa')
VISA.select(VISA['Name'], VISA['DNI']).show()

+--------------------+--------+
|                Name|     DNI|
+--------------------+--------+
|Francisco Arias S...|70000000|
|Norberto Marias Q...|50000000|
+--------------------+--------+



In [50]:
clientscards.groupBy("CardType").count().show()

+----------------+-----+
|        CardType|count|
+----------------+-----+
|American Express|    2|
|            Visa|    2|
|     Master Card|    1|
+----------------+-----+



In [70]:
clientscards.createOrReplaceTempView("clientestarjetas")

sqlDF = spark.sql("SELECT Name, DNI FROM clientestarjetas WHERE CardType='American Express' ")
sqlDF.distinct().show()

+-------------------+--------+
|               Name|     DNI|
+-------------------+--------+
|Arturo Belano Yañez|20000000|
+-------------------+--------+



#### Finalice el contexto de Spark

In [33]:
sc.stop()